# ——————L.3, T.1—————————

In [1]:
import pandas as pd
from scipy.stats import ttest_ind

In [2]:
data_df = pd.read_excel('final_attestation_data.xlsx', sheet_name='Dataset')
city_dict_df = pd.read_excel('final_attestation_data.xlsx', sheet_name='city_dict')

# ———————————————-
# table research

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21939 entries, 0 to 21938
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_view      21939 non-null  int64         
 1   time_view    21939 non-null  datetime64[ns]
 2   id_group     21939 non-null  int64         
 3   id_city      21939 non-null  int64         
 4   nflag_order  21939 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 857.1 KB


In [4]:
city_dict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_city    11 non-null     int64 
 1   name_city  11 non-null     object
dtypes: int64(1), object(1)
memory usage: 308.0+ bytes


# ———————————————-
# data merge

In [5]:
merged_data = data_df.merge(city_dict_df, on='id_city', how='left')

In [6]:
merged_data.head()

,id_view,time_view,id_group,id_city,nflag_order,name_city
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск
1,10457152884,2023-01-30 17:38:18.629,1,1,1,Москва
2,10457162947,2023-01-07 14:06:22.689,1,1,1,Москва
3,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск
4,10457176480,2023-01-04 22:56:19.240,0,1,1,Москва


# ——————L.3, T.2—————————

# ----------------------------------------------
# functions for calculation

In [7]:
def test_calc(r1, r2, alpha = 0.05):
    s, p = ttest_ind(r1, r2)

    if p < alpha:
        print("Гипотеза не подтверждается, средние не равны")
        print("Разница средних = ", r1.mean()-r2.mean())
    else:
        print("Гипотеза подтверждается, средние равны")
        print("Разница средних = 0")

    return s, p

# ----------------------------------------------
# calculating the overall result

In [8]:
group_A = merged_data[merged_data['id_group'] == 1]
group_B = merged_data[merged_data['id_group'] == 0]

conversion_A = group_A['nflag_order'].mean()
conversion_B = group_B['nflag_order'].mean()

print(f"Конверсия группы A: {conversion_A:.2f}\n")
print(f"Конверсия группы B: {conversion_B:.2f}\n")

print(test_calc(merged_data[merged_data['id_group'] == 1]['nflag_order'], merged_data[merged_data['id_group'] == 0]['nflag_order']))


Конверсия группы A: 0.68

Конверсия группы B: 0.62

Гипотеза не подтверждается, средние не равны
Разница средних =  0.060691822608150425
(9.41595134321716, 5.1365957325220535e-21)


# ——————L.3, T.3—————————

# ----------------------------------------------
# significant cities

In [9]:
significant_cities = [city_id for city_id in data_df['id_city'].unique()
                      if len(data_df[(data_df['id_group'] == 0) & (data_df['id_city'] == city_id)]) > 0
                      and len(data_df[(data_df['id_group'] == 1) & (data_df['id_city'] == city_id)]) > 0
                      and ttest_ind(data_df[(data_df['id_group'] == 1) & (data_df['id_city'] == city_id)]['nflag_order'],
                                    data_df[(data_df['id_group'] == 0) & (data_df['id_city'] == city_id)]['nflag_order'])[1] < 0.05]


city_info = {}
for city_id in significant_cities:
    city_name = city_dict_df[city_dict_df['id_city'] == city_id]['name_city'].values[0]
    group_A = data_df[(data_df['id_group'] == 1) & (data_df['id_city'] == city_id)]['nflag_order']
    group_B = data_df[(data_df['id_group'] == 0) & (data_df['id_city'] == city_id)]['nflag_order']
    stat, p_value = ttest_ind(group_A, group_B)
    city_info[city_id] = {'city_name': city_name, 'p_value': p_value, 's': stat}

print("Города со статистически значимыми различиями между группами A и B:\n")
for city_id, info in city_info.items():
    print(f"ID города: {city_id}, Название города: {info['city_name']}\n p-value: {info['p_value']}, s: {info['s']}\n")

Города со статистически значимыми различиями между группами A и B:

ID города: 1, Название города: Москва
 p-value: 1.368947706664987e-13, s: 7.410981603370934

ID города: 2, Название города: Санкт-Петербург
 p-value: 1.8595141124883944e-07, s: 5.220275988832596

ID города: 4, Название города: Казань
 p-value: 0.0023855694044274188, s: 3.0436700754076127



# ——————L.3, T.4—————————

In [10]:
peak_hours = [7, 8, 9, 10, 17, 18, 19, 20]
merged_data['peak_hour'] = merged_data['time_view'].apply(lambda x: 1 if x.hour in peak_hours else 0)


peak_group = merged_data[merged_data['peak_hour'] == 1]
non_peak_group = merged_data[merged_data['peak_hour'] == 0]

# ----------------------------------------------
# peak group

In [11]:
print("Результаты АВ-теста для пиковых заказов:\n")
print(test_calc(peak_group[peak_group['id_group'] == 1]['nflag_order'], peak_group[peak_group['id_group'] == 0]['nflag_order']))

Результаты АВ-теста для пиковых заказов:

Гипотеза не подтверждается, средние не равны
Разница средних =  0.07196282994729519
(6.422002058720224, 1.429452278435098e-10)


# ----------------------------------------------
# non peak group

In [12]:
print("Результаты АВ-теста для не-пиковых заказов:\n")
print(test_calc(non_peak_group[non_peak_group['id_group'] == 1]['nflag_order'], non_peak_group[non_peak_group['id_group'] == 0]['nflag_order']))

Результаты АВ-теста для не-пиковых заказов:

Гипотеза не подтверждается, средние не равны
Разница средних =  0.05513649587263003
(6.997243317124079, 2.7237344158581742e-12)


# ——————L.3, T.5—————————

###  По результатам, полученным в задании 3.2, гипотеза о равенстве средних не подтверждается. Это означает, что средние значения между группами A и B не равны, и есть статистически значимые различия в конверсии из просмотра в создание заказа. Это может указывать на то, что понижение цены на услугу за счёт новой платёжной механики вероятно оказывает влияние на поведение пользователей и их склонность создавать заказы после просмотра.

###  По результатам, полученным в задании 3.3, были отобраны три города, в которых гипотеза о равенстве средних не подтвердилась. Это означает, что новая платежная механика, вероятно, влияет на поведение пользователей в данных городах, и это изменение конверсии неслучайно.

### По результатам, полученным в задании 3.4, гипотеза о равенстве средних не подтверждается. Однако нельзя точно утверждать есть ли существенные отличия между пиковыми и не пиковыми часами.

### Таким образом, общий анализ АВ-теста показывает, что понижение цены на услугу оказывает существенное и статистическое влияние в создание заказа. Однако, решение о внедрении новой платежной механики следует принимать, учитывая и прочие факторы: экономическая целесообразность, потенциальные затраты на реализацию и внедрение новой механики, а также возможные риски и последствия.